## Preparation of data:


In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
# NLTK

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from bs4 import BeautifulSoup

# import sys
# !{sys.executable} -m pip install lxml

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varshanatarajan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/varshanatarajan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/varshanatarajan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
questions = pd.read_csv('../data/Questions.csv', encoding = "ISO-8859-1")
answers = pd.read_csv('../data/Answers.csv', encoding = "ISO-8859-1")
tags = pd.read_csv('../data/Tags.csv', encoding = "ISO-8859-1")

In [3]:
questions.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
questions.shape

(1264216, 7)

In [5]:
answers.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [6]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [7]:
questions_and_answers = questions.merge(answers, left_on=['OwnerUserId','Id'], right_on = ['OwnerUserId','ParentId'], how='left')

In [8]:
questions_and_answers = questions_and_answers.rename(columns = {'Id_x': 'QID',
                           'CreationDate_x': 'QuestionCreationDate', 
                           'Score_x': 'QuestionScore', 
                           'Body_x': 'QuestionBody', 
                           'CreationDate_y': 'AnswerCreationDate', 'Score_y': 'AnswerScore', 'Body_y': 'AnswerBody'})
questions_and_answers = questions_and_answers.drop(['Id_y', 'ParentId'], axis = 1)
questions_and_answers.head()

,QID,OwnerUserId,QuestionCreationDate,ClosedDate,QuestionScore,Title,QuestionBody,AnswerCreationDate,AnswerScore,AnswerBody
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,2008-08-01T16:09:47Z,12.0,<p>I wound up using this. It is a kind of a ha...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,NaN,NaN,NaN
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN,NaN,NaN
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,NaN,NaN,NaN
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,2008-08-01T23:49:57Z,4.0,"<p>Yes, I thought about that, but I soon figur..."


In [9]:
questions_and_answers.shape

(1265467, 10)

In [10]:
grouped_tags = tags.groupby('Id')['Tag'].apply(list)
print(grouped_tags)

Id
80                                [flex, actionscript-3, air]
90          [svn, tortoisesvn, branch, branching-and-merging]
120                                   [sql, asp.net, sitemap]
180         [algorithm, language-agnostic, colors, color-s...
260              [c#, .net, scripting, compiler-construction]
                                  ...                        
40143210                                     [php, .htaccess]
40143300                                    [google-bigquery]
40143340                            [android, android-studio]
40143360                                 [javascript, vue.js]
40143380                                  [npm, mocha, babel]
Name: Tag, Length: 1264216, dtype: object


In [11]:
dataset_with_tags = questions_and_answers.merge(grouped_tags, left_on=['QID'], right_on=['Id'], how='inner')

In [12]:
dataset_with_tags.shape

(1265467, 11)

In [13]:
data = dataset_with_tags.copy(deep=True)
rows = data.shape[0]
cols = data.shape[1]
print(f"Rows: {rows} and Cols: {cols}")
print(f"Sample data: {data.head()}")

Rows: 1265467 and Cols: 11
Sample data:    QID  OwnerUserId  QuestionCreationDate            ClosedDate  \
0   80         26.0  2008-08-01T13:57:07Z                   NaN   
1   90         58.0  2008-08-01T14:41:24Z  2012-12-26T03:45:49Z   
2  120         83.0  2008-08-01T15:50:08Z                   NaN   
3  180    2089740.0  2008-08-01T18:42:19Z                   NaN   
4  260         91.0  2008-08-01T23:22:08Z                   NaN   

   QuestionScore                                              Title  \
0             26  SQLStatement.execute() - multiple queries in o...   
1            144  Good branching and merging tutorials for Torto...   
2             21                                  ASP.NET Site Maps   
3             53                 Function for creating color wheels   
4             49  Adding scripting functionality to .NET applica...   

                                        QuestionBody    AnswerCreationDate  \
0  <p>I've written a database generation script i...

In [14]:
print("Total no of questions:",data["QID"].nunique())

Total no of questions: 1264216


## Data Cleaning:

In [19]:
print(f"Minimum Score: {data['QuestionScore'].min()}")
print(f"Maximum Score: {data['QuestionScore'].max()}")

min_score = data['QuestionScore'].min()
max_score = data['QuestionScore'].max()

(data['QuestionScore'] >= 1).value_counts()

Minimum Score: -73
Maximum Score: 5190


False    670615
True     594852
Name: QuestionScore, dtype: int64

In [212]:
min_score = data['AnswerScore'].min()
max_score = data['AnswerScore'].max()

(data['AnswerScore'] >= 0).value_counts()

False    1137249
True      128218
Name: AnswerScore, dtype: int64

In [20]:
grouped_tags = tags.groupby(['Id', 'Tag']).size().groupby(level=1).sum()
print("Count of each tag: ", grouped_tags)  # count of each tag

Count of each tag:  Tag
.a                13
.app              14
.aspxauth          6
.bash-profile     46
.class-file       19
                ... 
zwoptex            3
zxing            159
zxspectrum         1
zynq              11
zypper             2
Length: 37034, dtype: int64


In [21]:
q = set()
for sublist in data['Tag'].values:
    q.update(sublist)
print("No of unique tags: ", len(q))

No of unique tags:  37035


In [22]:
# data = actual_data.copy(deep = True)
actual_data = data.copy(deep=True)
all_tags_list = [item for row in data['Tag'] for item in row]
tags_counts = nltk.FreqDist(all_tags_list)
_50_most_common = tags_counts.most_common(50)
most_common_tags = [tag[0] for tag in _50_most_common]
print(f"Taking 50 most common - {most_common_tags}")

Taking 50 most common - ['javascript', 'java', 'c#', 'php', 'android', 'jquery', 'python', 'html', 'c++', 'ios', 'mysql', 'css', 'sql', 'asp.net', 'objective-c', 'ruby-on-rails', '.net', 'c', 'iphone', 'angularjs', 'arrays', 'sql-server', 'json', 'ruby', 'r', 'ajax', 'regex', 'xml', 'node.js', 'asp.net-mvc', 'linux', 'django', 'wpf', 'database', 'swift', 'xcode', 'string', 'excel', 'vb.net', 'windows', 'spring', 'wordpress', 'eclipse', 'html5', 'multithreading', 'oracle', 'git', 'facebook', 'forms', 'bash']


In [23]:
# filtering the dataset with only the most common tags
def most_common(tags):
    tags_to_send = []
    for t in tags:
        if t in most_common_tags:
            tags_to_send.append(t)
    return tags_to_send


data['Tag'] = data['Tag'].apply(lambda x: most_common(x))
data['Tag'] = data['Tag'].apply(lambda x: x if len(x) > 0 else None)
data['QuestionScore'] = data['QuestionScore'].apply(lambda x: x if x >= 3 else None)
data.dropna(subset='Tag' , inplace=True)
data.dropna(subset='QuestionScore', inplace = True)

In [25]:
print(data)

              QID  OwnerUserId  QuestionCreationDate            ClosedDate  \
2             120         83.0  2008-08-01T15:50:08Z                   NaN   
4             260         91.0  2008-08-01T23:22:08Z                   NaN   
5             330         63.0  2008-08-02T02:51:36Z                   NaN   
6             470         71.0  2008-08-02T15:11:47Z  2016-03-26T05:23:29Z   
7             580         91.0  2008-08-02T23:30:59Z                   NaN   
...           ...          ...                   ...                   ...   
1265074  40134650    6435187.0  2016-10-19T14:42:25Z  2016-10-19T16:35:29Z   
1265101  40135240    1844024.0  2016-10-19T15:06:43Z                   NaN   
1265107  40135400    3581788.0  2016-10-19T15:12:54Z                   NaN   
1265214  40137660    4350660.0  2016-10-19T17:12:05Z                   NaN   
1265333  40140260    6421277.0  2016-10-19T19:45:53Z                   NaN   

         QuestionScore                                         

In [26]:
print("From: ", actual_data["QID"].nunique())
print("To: ", data["QID"].nunique())

From:  1264216
To:  146290


In [27]:
l = [text for text in data['AnswerBody']]
print(l[11])

nan


In [28]:
data = data.drop(columns=['AnswerCreationDate', 'ClosedDate', 'QuestionCreationDate','OwnerUserId'])

In [29]:
data

,QID,QuestionScore,Title,QuestionBody,AnswerScore,AnswerBody,Tag
2,120,21.0,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN,NaN,"[sql, asp.net]"
4,260,49.0,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,4.0,"<p>Yes, I thought about that, but I soon figur...","[c#, .net]"
5,330,29.0,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,NaN,NaN,[c++]
6,470,13.0,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,NaN,NaN,[.net]
7,580,21.0,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,NaN,NaN,[sql-server]
...,...,...,...,...,...,...,...
1265074,40134650,3.0,To find the overlapping of two date ranges and...,<p>We have blocks with specific no. of date ra...,NaN,NaN,[java]
1265101,40135240,3.0,R ggplot2: Add means as horizontal line in a b...,<p>I have created a boxplot using ggplot2:</p>...,NaN,NaN,[r]
1265107,40135400,3.0,Get attribute's name by it's value,<p>I need to return the key with a known <code...,NaN,NaN,"[ruby-on-rails, ruby]"
1265214,40137660,3.0,Boost intrusive pointer,<p>I'm a little confused about boost's intrusi...,NaN,NaN,[c++]


In [39]:
grouped_answers = data.groupby('QID')['AnswerBody'].apply(list)
grouped_answers

QID
120                                                     [nan]
260         [<p>Yes, I thought about that, but I soon figu...
330                                                     [nan]
470                                                     [nan]
580                                                     [nan]
                                  ...                        
40134650                                                [nan]
40135240                                                [nan]
40135400                                                [nan]
40137660                                                [nan]
40140260                                                [nan]
Name: AnswerBody, Length: 146290, dtype: object

In [38]:
grouped_answers

QID
120                                                     [nan]
260         [<p>Yes, I thought about that, but I soon figu...
330                                                     [nan]
470                                                     [nan]
580                                                     [nan]
                                  ...                        
40134650                                                [nan]
40135240                                                [nan]
40135400                                                [nan]
40137660                                                [nan]
40140260                                                [nan]
Name: AnswerBody, Length: 146290, dtype: object

In [32]:
data_with_grouped_answers = data.merge(grouped_answers, left_on=['QID'], right_on=['QID'], how='left')
data_with_grouped_answers = data_with_grouped_answers.drop(columns=['AnswerBody_x'])

In [95]:
c = 0
for x in grouped_answers:
    if len(x) != 1 and pd.notna(x)[0]:
        c += 1
print('Total no of questions with answers: ', c)

Total no of questions with answers:  237


In [98]:
data.isna().sum()

data_with_grouped_answers.isna().sum() # This many null values are in the answer body

QID                   0
QuestionScore         0
Title                 0
QuestionBody          0
AnswerScore      130964
Tag                   0
AnswerBody_y          0
dtype: int64

In [99]:
data_with_grouped_answers

,QID,QuestionScore,Title,QuestionBody,AnswerScore,Tag,AnswerBody_y
0,120,21.0,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,NaN,"[sql, asp.net]",[nan]
1,260,49.0,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,4.0,"[c#, .net]","[<p>Yes, I thought about that, but I soon figu..."
2,330,29.0,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,NaN,[c++],[nan]
3,470,13.0,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,NaN,[.net],[nan]
4,580,21.0,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,NaN,[sql-server],[nan]
...,...,...,...,...,...,...,...
146590,40134650,3.0,To find the overlapping of two date ranges and...,<p>We have blocks with specific no. of date ra...,NaN,[java],[nan]
146591,40135240,3.0,R ggplot2: Add means as horizontal line in a b...,<p>I have created a boxplot using ggplot2:</p>...,NaN,[r],[nan]
146592,40135400,3.0,Get attribute's name by it's value,<p>I need to return the key with a known <code...,NaN,"[ruby-on-rails, ruby]",[nan]
146593,40137660,3.0,Boost intrusive pointer,<p>I'm a little confused about boost's intrusi...,NaN,[c++],[nan]


In [100]:
mask = pd.Series([np.nan not in lst for lst in data_with_grouped_answers.AnswerBody_y])
mask = mask[mask == True]
mask = mask.index.values.tolist()

data_with_grouped_answers['AnswerBody_y'] = data_with_grouped_answers.AnswerBody_y.str.join(sep=' ')

In [101]:
print(data_with_grouped_answers)

             QID  QuestionScore  \
0            120           21.0   
1            260           49.0   
2            330           29.0   
3            470           13.0   
4            580           21.0   
...          ...            ...   
146590  40134650            3.0   
146591  40135240            3.0   
146592  40135400            3.0   
146593  40137660            3.0   
146594  40140260            3.0   

                                                    Title  \
0                                       ASP.NET Site Maps   
1       Adding scripting functionality to .NET applica...   
2               Should I use nested classes in this case?   
3                   Homegrown consumption of web services   
4        Deploying SQL Server Databases from Test to Live   
...                                                   ...   
146590  To find the overlapping of two date ranges and...   
146591  R ggplot2: Add means as horizontal line in a b...   
146592                 Get att

In [102]:


data_with_grouped_answers['QuestionBody'].iloc[mask]  = data_with_grouped_answers['QuestionBody'].iloc[mask] + " " + data_with_grouped_answers['AnswerBody_y'].iloc[mask]



/var/folders/j2/_vj_24mj5vs43jytprthzk_r0000gn/T/ipykernel_85233/2841134462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_with_grouped_answers['QuestionBody'].iloc[mask]  = data_with_grouped_answers['QuestionBody'].iloc[mask] + " " + data_with_grouped_answers['AnswerBody_y'].iloc[mask]


In [103]:
data_with_grouped_answers = data_with_grouped_answers.drop(columns=['AnswerBody_y', 'AnswerScore'])

In [108]:
data_with_grouped_answers.isna().sum() # This many null values are in the body
data_with_grouped_answers = data_with_grouped_answers.rename(columns = {'QuestionBody': 'Body'})
data_with_grouped_answers

,QID,QuestionScore,Title,Body,Tag
0,120,21.0,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,"[sql, asp.net]"
1,260,49.0,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,"[c#, .net]"
2,330,29.0,Should I use nested classes in this case?,<p>I am working on a collection of classes use...,[c++]
3,470,13.0,Homegrown consumption of web services,<p>I've been writing a few web services for a ...,[.net]
4,580,21.0,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...,[sql-server]
...,...,...,...,...,...
146590,40134650,3.0,To find the overlapping of two date ranges and...,<p>We have blocks with specific no. of date ra...,[java]
146591,40135240,3.0,R ggplot2: Add means as horizontal line in a b...,<p>I have created a boxplot using ggplot2:</p>...,[r]
146592,40135400,3.0,Get attribute's name by it's value,<p>I need to return the key with a known <code...,"[ruby-on-rails, ruby]"
146593,40137660,3.0,Boost intrusive pointer,<p>I'm a little confused about boost's intrusi...,[c++]


In [110]:
# Converting html to text in the body

data_with_grouped_answers['Body'] = [BeautifulSoup(text).get_text() for text in data_with_grouped_answers['Body'] ]

In [376]:
print(data_with_grouped_answers)
print(data_with_grouped_answers.isna().sum())

             QID  QuestionScore  \
0            120           21.0   
1            260           49.0   
2            330           29.0   
3            470           13.0   
4            580           21.0   
...          ...            ...   
146590  40134650            3.0   
146591  40135240            3.0   
146592  40135400            3.0   
146593  40137660            3.0   
146594  40140260            3.0   

                                                    Title  \
0                                       ASP.NET Site Maps   
1       Adding scripting functionality to .NET applica...   
2               Should I use nested classes in this case?   
3                   Homegrown consumption of web services   
4        Deploying SQL Server Databases from Test to Live   
...                                                   ...   
146590  To find the overlapping of two date ranges and...   
146591  R ggplot2: Add means as horizontal line in a b...   
146592                 Get att

In [134]:
token = ToktokTokenizer()
punct = '!"$%&\'#()*+,./:;<=>?@[\\]^_`{|}~'
lemma = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))


def clean_text(text):
    text = text.lower()
    text = text.strip(' ')
    return text

def strip_list_noempty(mylist):
    newlist = (item.strip() if hasattr(item, 'strip') else item for item in mylist)
    return [item for item in newlist if item != '']

def clean_punct(text): 
    words = token.tokenize(text)
    punctuation_filtered = []
    regex = re.compile('[%s]' % re.escape(punct))
    tokens = [token for token in words if not re.match(re.compile('[%s]' % re.escape(punct)), token)]
    filtered_list = strip_list_noempty(tokens)
    return ' '.join(map(str, filtered_list))

def lemmatizeWords(text):
    words = token.tokenize(text)
    listLemma = []
    for w in words:
        x = lemma.lemmatize(w, pos="v")
        listLemma.append(x)
    return ' '.join(map(str, listLemma))

def stopWordsRemove(text):
    stop_words = set(stopwords.words("english"))
    words = token.tokenize(text)
    filtered = [w for w in words if not w in stop_words]
    return ' '.join(map(str, filtered))

In [111]:
new_question_df = data_with_grouped_answers.copy(deep=True)

In [137]:
# Remove stopwords, punctuation and lemmatize for text in body
new_question_df['Body'] = new_question_df['Body'].apply(lambda x: clean_text(x)) 
new_question_df['Body'] = new_question_df['Body'].apply(lambda x: clean_punct(x)) 
new_question_df['Body'] = new_question_df['Body'].apply(lambda x: lemmatizeWords(x))
new_question_df['Body'] = new_question_df['Body'].apply(lambda x: stopWordsRemove(x))

# # Remove stopwords, punctuation and lemmatize for title.
new_question_df['Title'] = new_question_df['Title'].apply(lambda x: str(x))
new_question_df['Title'] = new_question_df['Title'].apply(lambda x: clean_text(x)) 
new_question_df['Title'] = new_question_df['Title'].apply(lambda x: clean_punct(x)) 
new_question_df['Title'] = new_question_df['Title'].apply(lambda x: lemmatizeWords(x)) 
new_question_df['Title'] = new_question_df['Title'].apply(lambda x: stopWordsRemove(x))

In [140]:
df = new_question_df

## EDA: 